# Bitcoin Script Basics

## Reading 
- https://en.bitcoin.it/wiki/Script
- A full list of OP_CODES can be found [here](https://en.bitcoin.it/wiki/Script).


## Script debugger

This website (https://wschae.github.io/build/editor.html) is great for visualizing how bitcoin scripts get evaluated. For each transaction type we go through underneath, you can copy and paste the final script (scriptSig + scriptPubkey) into the debugger and click through step by step to see how it gets evaluated. You'll need copy and paste the hex string into the 'Assembly' tab for the online editor to parse it correctly. 

*Note that there are some [known issues](https://github.com/siminchen/bitcoinIDE/issues) with the debugger, particularly around the hash op codes.*

Here is an example using the script debugger to go through the example P2PKH script in the section below.
![script_debugger](../images/script-debugger.gif "script_debugger")


## Pushdata operations vs Variable-length integers

### Reading
- https://github.com/bitcoin/bips/blob/master/bip-0062.mediawiki#Push_operators
- https://bitcoin.stackexchange.com/questions/35904/how-much-of-bip-62-dealing-with-malleability-has-been-implemented
- https://bitcoin.stackexchange.com/questions/101817/is-there-a-difference-between-op-pushbytes-and-op-pushdata

### Variable-length integers
Bitcoin Script opcodes, and P2P variable-length integers are distinct concepts. In transactions, the field that encodes the number of inputs, number of outputs, length of the scriptPubkey or length of the scriptSig are variable-length integers. In our scripts we'll use the function `varint_len` to return the length of the data as a variable-length integer. For simplicity it'll only handle lengths up to `0xffff` (65535) as that'll cover our use cases. 


In [1]:
def varint_len(data: bytes):
    '''returns the length of the input as a variable integer'''
    l = len(data)
    if l < int('fd',16):
        varint = l.to_bytes(1, byteorder="little", signed=False)
    elif l < int('ffff',16):
        varint = bytes.fromhex("fd") + l.to_bytes(2, byteorder="little", signed=False)
    else:        
        raise Exception("This function only handles up to 0xffff bytes")
    return varint

### Pushdata operations
In bitcoin scripts, any data pushed onto the stack must be preceeded by a push operation. Since there are many push op codes, there are many potential ways to achieve the same end result. However, standardness rules dictate that the most concise method must be used. The most concise method for pushing data onto the stack is described in [BIP62](https://github.com/bitcoin/bips/blob/master/bip-0062.mediawiki#Push_operators). For our scripts we'll use the function `pushbytes` to prepend the appropriate push operations to the data to be pushed on to the stack. Note that this function does not cover large lengths, but will suffice for our scripts.

#### Notation
When notating scripts, data that is to be pushed onto the stack are generally enclosed in angled brackets e.g. `<pubkey hash>`, and the data push OP_CODES are ommitted.

For example, the following script (taken from a p2pkh scriptPubkey) can be written in either of the following ways.
- omitting OP_PUSHBYTES: `OP_DUP` `OP_HASH160` `<pubkey_hash>` `OP_EQUALVERIFY` `OP_CHECKSIG`
- including OP_PUSHBYTES:`OP_DUP` `OP_HASH160` `OP_PUSHBYTES20` `[20_byte_hash]` `OP_EQUALVERIFY` `OP_CHECKSIG`
- hex: `76` `a9` `14` `fc7250a211deddc70ee5a2738de5f07817351cef`  `88` `ac`

In [2]:
def pushbytes(data: bytes):
    '''prepends the length of the input in bytes. 
    Used for adding OP_PUSHBYTES in bitcoin script where stack items can be of arbitrary length.
    see BIP62
    '''
    l = len(data)
    if l <= 76:
        pushbytes = l.to_bytes(1, byteorder="little", signed=False)
    elif l <= 255:
        pushbytes = bytes.fromhex("4c") + l.to_bytes(1, byteorder="little", signed=False)
    elif l <= 520:
        pushbytes = bytes.fromhex("4d") + l.to_bytes(2, byteorder="little", signed=False)
    else:        
        raise Exception("This function only handles up to 520 bytes")
    return pushbytes + data

## P2PKH

scriptPubkey: `OP_DUP` `OP_HASH160` `<pubkey hash>` `OP_EQUALVERIFY` `OP_CHECKSIG`


scriptSig: `<pubkey>` `<signature>`

### Example serialized script

scriptPubkey hex: `76` `a9` `14` `fc7250a211deddc70ee5a2738de5f07817351cef` `88` `ac`

scriptSig hex: `48` `3045022100fda17d7d49101c7eaa8c2ed98a434d1098584fd780577e989b52fbf077a6fd8b0220050305f4cf3013026004062e75074969963b9b6fecae905980e5616b876bde9d01` `21` `034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa`

scriptSig followed by the scriptPubkey:
`483045022100fda17d7d49101c7eaa8c2ed98a434d1098584fd780577e989b52fbf077a6fd8b0220050305f4cf3013026004062e75074969963b9b6fecae905980e5616b876bde9d0121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa76a914fc7250a211deddc70ee5a2738de5f07817351cef88ac`

## P2SH

- [BIP16 - Pay to Script Hash](https://github.com/bitcoin/bips/blob/master/bip-0016.mediawiki)

Here is the general format of the scriptPubkey and scriptSig for a P2SH transaction. The section in the scriptSig marked `...` is the part of the script which should unlock the redeemScript. e.g. For a 2-of-3 multisig it would be two signatures, or for a hashlock it would be the hash preimage.


scriptPubkey: `OP_HASH160` `<redeem_script_hash>` `OP_EQUAL`

scriptSig: ` ... <redeem_script>`

### P2SH (m of n multisig)

- [BIP11 - M-of-N Standard Transactions](https://github.com/bitcoin/bips/blob/master/bip-0011.mediawiki)

The first number 'm' (in this example `OP_2`) represents the number of signatures needed, the second number 'n' (here `OP_3`) represents the total number of pubkeys. The order of the signatures in the scriptSig must be in the same order as the pubkeys in the redeemScript. 

Note that it is possible to create a transaction with the scriptPubkey encoding the multisig logic directly, known as 'bare multisig'. However, a transaction with such an output would make it 'non-standard'.

Here is an example of a 2 of 3 multisig.

scriptPubkey: `OP_HASH160` `<redeem_script_hash>` `OP_EQUAL`

redeemScript : `OP_2` `<pubkey1>` `<pubkey2>` `<pubkey3>` `OP_3` `OP_CHECKMULTISIG>`

scriptSig: `OP_0` `<signature1>` `<signature2> <redeemScript>`

### Example serialized script

scriptPubkey hex: `a9 14 19130817a355e1a4df9cb1e25052d39374b83be8 87`

scriptSig hex: `00 48 30450221008f728a9536c70fd0bb162139e0a248fc79149710be701f1320e61bcd347e45ca02204144397f7f086efcee8d0c8ddebfca74bb5511b71cf7e15b6d4dae355b08f84d01 48 3045022100e620ce1fe7523edaf1e0c757df8f347999251475f1060a37a92556fb3aa8904b02200378a0795d2665c94d58f95f3621086035147299b13c0640f7f53845deae3a3701 4c 69 52 21034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa 2102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f27 21023c72addb4fdf09af94f0c94d7fe92a386a7e70cf8a1d85916386bb2535c7b1b1 53 ae`


scriptSig followed by the scriptPubkey (note that this script is not evaluated properly by the online bitcoin script debugger):
`004830450221008f728a9536c70fd0bb162139e0a248fc79149710be701f1320e61bcd347e45ca02204144397f7f086efcee8d0c8ddebfca74bb5511b71cf7e15b6d4dae355b08f84d01483045022100e620ce1fe7523edaf1e0c757df8f347999251475f1060a37a92556fb3aa8904b02200378a0795d2665c94d58f95f3621086035147299b13c0640f7f53845deae3a37014c695221034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa2102466d7fcae563e5cb09a0d1870bb580344804617879a14949cf22285f1bae3f2721023c72addb4fdf09af94f0c94d7fe92a386a7e70cf8a1d85916386bb2535c7b1b153aea91419130817a355e1a4df9cb1e25052d39374b83be887`

### Quiz
Given these locking scripts, which scriptSigs would allow the output to be spendable?
Q1. `02 OP_ADD 03 OP_EQUALVERIFY`
- A. `01`
- B. `02`
- C. `03`
- D. `04`

A: `01`

Q2. `OP_DUP OP_ADD 08 OP_EQUALVERIFY`
- A. `03`
- B. `04` 
- C. `05`
- D. `06`

B: `04`